# **Sistema Lineares**
---
<ul>
  <li><a href="#scrollTo=6T7Gy-Docn5C&uniqifier=1">Sistemas Lineares</a>
  </li>
  <ul>
      <li><a href="#scrollTo=dRkBMuF7ZtwR&uniqifier=1">Introdução</a></li>
    <li><a href="#scrollTo=RoOsu9Ppsd0v&uniqifier=1">Funções comuns</a></li>
    <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Método direto</a>
      <ul>
          <li><a href="#scrollTo=fvXmmrPMwIVa&uniqifier=1">Métodos</a></li>
          <li><a href="#scrollTo=Q1l9-tNWhuaq&uniqifier=1">Comparações</a></li>
      </ul>
    </li>
    <li><a href="#scrollTo=Ti2W5pc9VLcl&uniqifier=1">Método iterativo</a>
        <ul>
          <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Métodos</a></li>
          <li><a href="#scrollTo=Q1l9-tNWhuaq&uniqifier=1">Comparações</a></li>
      </ul>
    </li>
    <li><a href="#scrollTo=BP3sHmVDk-s1&uniqifier=1">Aplicações</a></li>
  </ul>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
from scipy import random

# Introdução

> $Ax = b$

"O vetor $b_m$ pode ser expresso como combinação linear das colunas de $A_{mn}$?"[1] 

> $a_{11}*x_{1} + \cdots + a_{1n}*x_{n} = b_{1}$
>
> $a_{21}*x_{1} + \cdots + a_{2n}*x_{n}= b_{2}$
>
> $\vdots$ 
>
> $a_{m1}*x_{1} + \cdots + a_{mn}*x_{n} = b_{m}$


# Funções Comuns

In [3]:
import time

# @Decorator timeit
# Retorna o tempo de excução da função
# Não modifica a função.
# Para mais informações sobre decorators e timeit:
# https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__)
            kw['log_time'][name].append((te - ts) * 1000)
        else:
            print("%r  %2.5f ms" % (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [4]:
# - @function plot_stat_methods.
# - @param    *     : obriga o usuário a nomear o parametro quando chamar a função.
# - @param function : method1 primeiro metodo.
# - @param functon  : method2 segundo metodo.
# - @param string   : namef1 nome do primeiro metodo.
# - @param string   : namef2 nome do segundo metodo.
# - @param number   : maxn maximo tamanho da matriz.
# - @param number   : increment é o passo de testes, que vai de 10 até um número menor que n.
# - @param number   : e é o erro minimo.
# - @param string   : tmatrix é o tipo de matriz que será usada.
#   Plota o tempo e erro de dois métodos.
def plot_stat_methods(*, method1, method2, namef1, namef2, maxn = 600, increment = 100, e = 1e-3, tmatrix = 'random'):
  # Tempo gerado por execução
  logtime_data = {
     namef1 : [],
     namef2 : [],
  }

  # erro gerado por execução
  logerror_data = {
    namef1 : [0.09, 0.08, 0.03, 0.04, 0.08, 0.05, 0.03, 0.021],
    namef2 : [0.009, 0.8, 0.03, 0.004, 0.08, 0.05, 0.07, 0.091],
  } 

  x = [];
  for i in range(10, maxn, increment):
    method1(i, log_time=logtime_data)
    method2(i, log_time=logtime_data)
    x.append(i);

  fig = plt.figure(figsize=(12,6));
  ax1 = fig.add_subplot(111)

  # Plot time
  ax1.plot(x, logtime_data.get(namef1), 'b', label= namef1 + " time",  linewidth=3.5)
  ax1.scatter(x, logtime_data.get(namef1), color='b')

  ax1.plot(x, logtime_data.get(namef2), 'y', label= namef2 + " time",  linewidth=3.5)
  ax1.scatter(x, logtime_data.get(namef2), color='y')

  # Sobre twinx: https://matplotlib.org/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-pyhttps://matplotlib.org/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-py
  ax2 = ax1.twinx()
  color = 'tab:red'
  
  # Plot errors
  if( len(logerror_data.get(namef2)) > 0 ):
      ax2.plot(x, logerror_data.get(namef2), 'g--',  linewidth=0.8)
      ax2.scatter(x, logerror_data.get(namef2), label= namef2 + ' errors', marker='o', color='g')

  if( len(logerror_data.get(namef1)) > 0 ):
    ax2.plot(x, logerror_data.get(namef1), 'r--', linewidth=0.8)
    ax2.scatter(x, logerror_data.get(namef1), label= namef1 + ' errors', marker='o', color='r')

  # legendas e axes
  ax1.set(title="Tempo(s) de execução em função de n", xlabel="Tamanho n", ylabel="Tempo(s)")
  ax2.set_ylabel('Erros', color=color)
  ax1.legend(loc="best", fontsize='large')
  ax2.legend(loc="best", fontsize='large')

  plt.show()

In [ ]:
def get_special_spd():
  A = np.zeros((n, n)) #cria matriz com tudo zero
  np.fill_diagonal(A[:, :], 6)      #diagonal principal
  np.fill_diagonal(A[1:, :-1], -4)  #digonal abaixo da principal
  np.fill_diagonal(A[:-1, 1:], -4)  #diagonal acima da principal
  np.fill_diagonal(A[:-2, 2:], 1)   #diagonal duas abaixo da principal
  np.fill_diagonal(A[2:, :-2], 1)   #diagonal duas cima da principal

  # Alguns valores distintos
  A[0, 0] = 9
  A[n-1, n-1] = 1
  A[n-2, n-2] = 5
  A[n-2, n-1] = -2
  A[n-1, n-2] = -2
  return A  

## Tipo de matrizes e propriedades

Propriedades:

Se a matriz não é singular, seu determinante determinantes é diferente de zero, então sabemos que há pelo menos uma solução.
> $x = -A^{-1}*b$



In [10]:
def generate_matrix(n=3, tmatrix='random'):
    if(tmatrix=='special_spd'):
      get_special_spd()

    A =  np.random.rand(n, n)
    if(tmatrix=='semi-spd'):
      return np.dot(A, np.transpose(A))
    elif(tmatrix=='spd'):
      return np.dot(A, np.transpose(A))
    else:
      return A
   


**Foward Solution Ly = b**

>$L_{m,n} =
 \begin{pmatrix}
  l_{1,1} & 0 & \cdots & 0 \\
  l_{2,1} & l_{2,2} & \cdots & 0 \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  l_{m,1} & l_{m,2} & \cdots & l_{m,n}
 \end{pmatrix}b_{m} =
 \begin{pmatrix}
  b_{1} \\
  b_{2} \\
  \vdots  \\
  b_{m}
 \end{pmatrix}$


> $y_1 = b_1/l_{11}$

>$\vdots$

> $y_{n} = b_n - \sum_{k=1}^{n} l_{ik}*y_{ik}$ 

In [5]:
# recebe a matriz inferior triangular A, e o vetor b. Retorna o vetor solução y.
def forward_substitution(A, b):
  l = np.copy(A);
  y = np.copy(b); 
  n = np.shape(A)[0];

  for i in range(1, n):
    y[i] -= np.dot(l[i, 0:i], y[0:i]);
  return y;

**Back Solution Ux = y (then)**

>$U_{m,n} =
 \begin{pmatrix}
  u_{1,1} & u_{1,2} & \cdots & u_{1,n} \\
  0 & a_{2,2} & \cdots & u_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  0 & 0 & \cdots & u_{m,n}
 \end{pmatrix}
y_{m} =
 \begin{pmatrix}
  y_{1} \\
  y_{2} \\
  \vdots  \\
  y_{m}
 \end{pmatrix}$

> $x_n = y_n/u_{nn}$

> $x_{n-1} = (y_{n-1}-x_{n}*u_{n-1n})/u_{n-1n-1}$ 


In [ ]:
# Recebe a matriz tringular superiror A e o vetor y, Ux = y, returna o vetor solução x
def back_substituion(A, y): 
  u = np.copy(A);
  x = np.copy(y);
  n = np.shape(A)[0];

  for i in range(n-1, -1, -1):
    x[i] = (x[i] - np.dot(u[i,i+1:n], x[i+1:n]))/u[i, i];

  return x;

O determinante da matriz U (triangular superior) é igual ao produto da diagonal.
$|A| = |U| = \prod_{i=1}^n u_{ii}$

In [ ]:
# Recebe U, a matriz triangular superior quadrada, retorna o determinante (O produto da diagonal) 
def determinante_upper_matrix(U):
  return U.diagonal().prod();

# **Métodos diretos**


**Solve Ax = b**

> $Ax = (LU)x = b$
>
> $L(Ux) = b$
>
> $Ly = b$    // foward solution
>
> $Ux = y$    // back solution

In [ ]:
# Recebe a matriz A, o vetor b e a função de decomposição A = (LU)
def lu_solve_system(A, b, decomposition):
    L, U = decomposition(A);
    y = forward_substitution(L, b);
    x = back_substituion(U, y);
    return x

## **O método de Gaus**

O método de Gaus consiste em obter a matriz triangular inferior utilizando apenas operações elementares na forma matricial.

Elas são, sobre a matriz A:

1. Tocar duas linhas de A, o que muda o sinal de |A|.
2. Multiplicar uma linha por um escalar $a$, diferente de zero, o que implica $a|A|$.
3. Multiplicar uma linha e subtrair de uma linha. 

Para eliminar um termo $A_{i+kj}$, usamos o termo $A_{ij}$, em que $k = i + 1, ... n$

> $a = A_{i+1j} / A_{ij}$
>
> $A_{i+1j} = A_{i+1j} - a*A_{ij} = 0$
>

Operação para modificar uma coluna:
> $A_{ij} = A_{ij} +- a * A_{kj}, j = k, k + 1, ..., n$
>
> $b_i = b_i - a*b_k$

Com A transformada em U, agora podemos fazer:
> $Ux = y$



## **O método da decomposição LU**

Decompoẽ a matriz A em LU. Multiplicando a forma idela de L e U obtem-se a seguinte expressão:

>$A_{m,n} =
 \begin{pmatrix}
  u_{1,1} & u_{1,2} & u_{1,3} \\
  u_{1,1}*l_{2,1} & u_{1,2}*l_{2,1} + u_{2,2}  & u_{1,3}*l_{2,1} + u_{2,3} \\
  u_{1,1}*l_{3,1} & u_{12}*l_{3,1} + u_{2,2}*l_{3,2} &  u_{13}*l_{3,1} + u_{2,3}*l_{3,2} + u_{3,3}
 \end{pmatrix}$

Assim, podemos notar que de $ k = 0...n-1$ e $i = k + 1,..., n$ os termos $a_{ik}/a_{kk} = l_{ik}$ e aplicando a eliminação de Gauss obteremos a matriz triangular superior U.

In [ ]:
# Recebe a matriz A, retorna a matriz aumentada [L/U]
def LU_decomposition(A):
  LU = np.copy(A);
  n = len(A);

  for k in range(0, n-1):
    for i in range(k+1, n):
      if(A[i, k] != 0):
        m = A[i, k]/A[k, k]
        A[i, k+1:n] -= m*A[k, k+1:n];
        A[i, k] = m;

  return A

## **O método de Gaus com pivoteamento**

O pivoteamento é uma forma de preprocessar a matriz antes do método de Gauss devido aos dois seguintes casos:

>$A_{m,n} =
 \begin{pmatrix}
  0 & -1 & 1 & | 0\\
  -1 & 2 & -1 & | 0 \\
  2 & -1 & 0 & |  1 \\
 \end{pmatrix}$
>$A_{m,n} =
 \begin{pmatrix}
  e & -1 & 1 & | 0\\
  -1 & 2 & -1 & | 0 \\
  2 & -1 & 0 & |  1 \\
 \end{pmatrix}$

Em que $1 > e > -1$.

O pivoteamento reorganiza a matriz visando obter a matriz mais proxima com diagonal dominante, a qual pode ser definida como a matriz que tem a soma das linhas (sem contar elementos da diagonal principal) menor que o elemento diagonal. 

No algoritmo abaixo, o melhor pivor é escolhido em relação ao maior tamanho relativo. O tamanho relativo é o valor do elemento normalizado pelo maior elemento contido na mesma linha: $r_{ij} = \max_j (|A_{ij}| / max(A_{i}), j \geq k$


In [ ]:
# Recebe a matriz A, os indexes i e j, referente as duas linhas que comutarão.
def swap_row(A, i, j):
  if(len(A.shape) == 1):
    A[i], A[j] = A[j], A[i]
  else:
    tmp = np.copy(A[i, :])
    A[i, :] = np.copy(A[j, :])
    A[j, :] = np.copy(tmp[:])

In [ ]:
# Recebe a matriz A, e o vetor b. Returna [U|y] (matriz aumentada)
def gauss_pivot(A, b, tol=1.0e-9):
  U = np.copy(A);
  y = np.copy(b);
  n = len(b);

  s = np.zeros([n])

  # escolhe-se os maiores elementos de cada linha.
  for i in range(0, n):
    s[i] = np.max(np.abs(A[i, :]))
  
  # pivoteamento
  for k in np.arange(n-1):
    # A próxima linha é aquela que possivel o elemento com maior tamnho relativo. 
    p = np.argmax(np.abs(A[k, k:n]/s[k:n]))
    if(np.abs(A[p, k]) < tol):
      error.err('Matriz singular!')
    if(p != k):
      swap_row(A, p, k)
      swap_row(b, p, k)
      swap_row(s, p, k)

    # eliminação de Gauss
    for i in range(k+1, n):
      m = 0.0;
      if(A[i, k] != 0):
        m = A[i, k]/A[k, k]
        A[i, k+1:n] -= m*A[k, k+1:n];
        b[i] -= m*b[k]; 

  return [A, b]

## **O método de Cholesks**

Esse método decompoẽ uma matriz simétrica e positiva definida na matriz L.

> $A = LL^t = LU => U = L^t$

Se $i = j$

> $L_{jj} = \sqrt (A_{jj} -\sum_{k=1}^{j-1} L_{j,k}^2), j = 2, 3, ..., n$

Se $i \neq j$

> $L_{ij} = \sqrt (A_{ij} -\sum_{k=1}^{j-1} L_{i,k}*L_{j,k}) / L_{jj}, j = 2, 3, ..., n$

In [ ]:
# Recebe a matrix simetrica, positiva definida A = L @ (^t)
def choleskis_decomposition(a):  
  L = np.copy(A);
  n = len(a);

  for i in range(0, n):
    try:
      a[i, i] = np.sqrt(a[i, i]-np.dot(a[i, 0:i], a[i, 0:i]))
    except ValueError:
      error.err('Not a define posit]ive Matrix')  
    for k in range(i+1, n):
      a[k,i] = (a[k,i] - np.dot(a[k, 0:i], a[i, 0:i]))/a[i,i]
  for k in range(1, n):
    a[0:k, k] = 0.0
  return a

In [ ]:
def choleskis_solve(A, b):
  L = choleskis_decomposition(np.copy(A))
  U = transverse(np.copy(L))
  y = forwardSubstitution(L, b)
  return backSubstituion(U, y)


## **Sistemas lineares diretos: Comparativo entre os métodos**

In [ ]:
A = np.array([[2, 3], [5, 4]]);
b = np.array([8, 13]);
ans = np.array([1, 2]);

NameError: ignored

# **Métodos iterativos**


## **Gauss-Seidel**


In [ ]:
def gauss_seidel(x, n, tol=1.0e-9):
  w = 1.0
  p = 1
  k = 10
  for i in range(1,n):
    xOld = np.copy(x)
    x = interEqs(x, w)
    dx = np.sqrt(np.dot(x-xOld, x-xOld))
    if(dx < tol):
      return [x, i]
    if(i == k):
      dx1 = dx
    if(i == k + p):
      dx2 = dx
      w = 2.0/(1.0 + np.sqrt(1.0-(dx2/dx1)))

  print("No converge for ", tol)
  return  [-1]

## **Gauss-Jacobi**




In [ ]:
# System Linear Solve with relatixion omega(w)
def inter_eqs_jacobini(x, w = 1.0):
  x1 = np.zeros(len(x))
  for i in range(len(x)):
    x1[i] = (w*(b[i] - np.dot(a[i,:], x[:]) + np.dot(a[i, i], x[i]))/a[i,i])-((1-w)*x[i])
  return x1

In [ ]:
#
def gauss_jacobi(x, n, tol=1.0e-9):
  for i in range(1,n):
    xOld = np.copy(x)
    x  = inter_eqs_jacobini(x)
    dx = np.sqrt(np.dot(x-xOld, x-xOld))
    if(dx < tol):
      return [x, i]
  return [-1]

## Aplicações

## Calculo de densidades

![Provetas](https://www.researchgate.net/publication/321584117/figure/fig5/AS:569090473709574@1512693150597/Figura-5-Etapas-de-teste-de-proveta-e-regioes-formadas-durante-a-sedimentacao-FAUST.png)

Figura: imagem da analise de acumulo de sedimentos

A partir da relação $massa(m) = densidade(d) / volume(v)$, queremos descobrir a densidade $d_a$, $d_b$, $d_c$, $d_d$ dos respectivos liquidos A, B, C e D.

Porém apenas temos como estimar os volumes dos liquidos. Por exemplo, o liquido B na proveta 1, $v_{1b}$, na proveta 2, $v_{2b}$, do liquido A na proveta 2, $v_{2a}$, e assim adiante.

Para cada proveta, também sabemos as massas contidas ali dentro, desconsiderando as massas das provetas, sendo estas respectivamente, $m_1$, $m_2$, $m_3$ e $m_4$. Com essas informações, temos o seguinte sistema:

> $v_{1a}*d_{a} + \cdots + v_{1d}*d_{d} = m_{1}$
>
> $v_{2a}*d_{a} + \cdots + v_{2d}*d_{d}= m_{2}$
>
> $v_{3a}*d_{a} + \cdots + v_{3d}*d_{d} = m_{3}$
>
> $a_{4a}*d_{a} + \cdots + v_{4d}*d_{d} = m_{4}$



## Representação das cores

Podemos representar as cores como a adicação entre R(Red), G(Green), B(Blue), sendo a menor intensidade de cada cor é 0 e a maior é 1.

A representação da cor $i$ então será: $C_i = (R_i, G_i, B_i) = R_i(1, 0, 0) + G_i(0, 1, 0) + B_i(0, 0, 1)$.

Defini-se também que a quantidade da cor gerada é no máximo 1 e assim as cores que forma $C_i$ não devem exceder essa quantidade: $x_a + x_b + x_c = 1$.

Uma mistura entre três cores pode ser representada como:
$C = (C_1, C_2, C_3) = (1/4, 2/4, 1/4)$ e teremos o seguinte sistema linear:

> $R_1*x_a + R_2*x_b + R_3*x_c = 1/4$
>
> $G_1*x_b + G_2*x_b + R_3*x_c = 2/4$
>
> $B_1*x_c + B_2*x_b + R_3*x_c = 1/4$
>
> $x_a + x_b + x_c = 1$




# Referências

[1] Scientific Computing An Introductory Survey, capitulo 2, System of Linear Equations.

[2] IME curso Calculo Númerico IAG-2005, resumo colli, capitulo 1, 2, 3.

[3] Datacamp, tutorial matplotlib, Julho 10, 2020. Disponivel em: https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python